In [3]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import scipy.sparse 
from matplotlib import pyplot as plt

from transformers import AutoTokenizer, AutoModel
import tensorflow_hub as hub
 
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from tqdm import tqdm

## Load Data

In [4]:
ekg_denoised = pd.read_pickle('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_v2.pkl')
ekg_denoised = ekg_denoised.sample(frac=1, random_state=10)


In [5]:
df_train, df_val, df_test = np.split(ekg_denoised.sample(frac=1, random_state=42), 
                                     [int(.8*len(ekg_denoised)), int(.9*len(ekg_denoised))])

print(len(df_train),len(df_val), len(df_test))

54544 6818 6818


## CORe PyTorch Model

### Generate classes for pytorch

In [6]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label for label in df['ICD9_CODE']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                truncation=True,
                                return_tensors="pt") 
                      for text in df['TEXT']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [7]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.softmax(linear_output)

        return final_layer



In [8]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

### Counting parameters

In [11]:
my_nn = BertClassifier()
pytorch_total_params = sum(p.numel() for p in my_nn.parameters() if p.requires_grad)


In [12]:
pytorch_total_params

108313348

In [17]:
print(my_nn)

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

### Hyperparameters and Training

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-outcome-biobert-v1", model_max_length=264)
BertModel = AutoModel.from_pretrained("bvanaken/CORe-clinical-outcome-biobert-v1")
model = BertClassifier()
EPOCHS = 2
LR = 1e-6

In [8]:
train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 27272/27272 [23:27<00:00, 19.38it/s]


Epochs: 1 | Train Loss:  0.676                 | Train Accuracy:  0.334                 | Val Loss:  0.667                 | Val Accuracy:  0.359


100%|██████████| 27272/27272 [24:22<00:00, 18.64it/s]


Epochs: 2 | Train Loss:  0.663                 | Train Accuracy:  0.376                 | Val Loss:  0.665                 | Val Accuracy:  0.369


### Testing

In [9]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.372


### Training 2 more epochs

In [12]:
train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 27272/27272 [24:36<00:00, 18.48it/s]


Epochs: 1 | Train Loss:  0.656                 | Train Accuracy:  0.397                 | Val Loss:  0.662                 | Val Accuracy:  0.379


100%|██████████| 27272/27272 [24:37<00:00, 18.46it/s]


Epochs: 2 | Train Loss:  0.647                 | Train Accuracy:  0.419                 | Val Loss:  0.664                 | Val Accuracy:  0.370


In [13]:
evaluate(model, df_test)

Test Accuracy:  0.371


Three epochs appears to work best for this model